In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np
import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [ ]:
# Установка случайного состояния
RANDOM_STATE = 42


In [ ]:

# 1. Загрузка данных
# Замените путь на ваш CSV-файл или DataFrame
data_path = '/content/OCLAR - Opinion Corpus for Lebanese Arabic Reviews.csv'
df = pd.read_csv(data_path)

In [ ]:
df.head()

,pagename,review,rating
0,Beirut Golden Plaza Suites,هذا الفندق ينقصه بعض الاشياء داخل الغرف مثلا ع...,2
1,Beirut Golden Plaza Suites,لطيف ولكن الغرف الفندقية تحتاج صيانة كادر الخد...,4
2,Beirut Golden Plaza Suites,مكان جميل جدا وحسن الخلق والضيافه,5
3,Beirut Golden Plaza Suites,بحاجة الى اعادة تأهيل للمفروشات,3
4,Beirut Golden Plaza Suites,فندق ممتاز ومعاملة راقية جدا,5


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3916 entries, 0 to 3915
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pagename  3916 non-null   object
 1   review    3916 non-null   object
 2   rating    3916 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 91.9+ KB


In [ ]:
# 2. Предобработка текста
def preprocess_text(text):
    # Приводим к нижнему регистру
    text = str(text).lower()

    # Удаляем знаки пунктуации
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Удаляем цифры и символы
    text = re.sub(r'\d+', '', text)

    # Удаляем лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [ ]:
# Применяем предобработку ко всем отзывам
df['clean_review'] = df['review'].apply(preprocess_text)

In [ ]:
# Проверяем результат
print(df[['review', 'clean_review']].head())

                                              review  \
0  هذا الفندق ينقصه بعض الاشياء داخل الغرف مثلا ع...   
1  لطيف ولكن الغرف الفندقية تحتاج صيانة كادر الخد...   
2                  مكان جميل جدا وحسن الخلق والضيافه   
3                    بحاجة الى اعادة تأهيل للمفروشات   
4                       فندق ممتاز ومعاملة راقية جدا   

                                        clean_review  
0  هذا الفندق ينقصه بعض الاشياء داخل الغرف مثلا ع...  
1  لطيف ولكن الغرف الفندقية تحتاج صيانة كادر الخد...  
2                  مكان جميل جدا وحسن الخلق والضيافه  
3                    بحاجة الى اعادة تأهيل للمفروشات  
4                       فندق ممتاز ومعاملة راقية جدا  


In [ ]:
 #3. Разделение данных на признаки и метки
X = df['clean_review']        # признаки (тексты)
y = df['rating']              # метки (оценки)

In [ ]:
# 4. Преобразование текста в TF-IDF вектора
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))  # биграммы тоже
X_tfidf = vectorizer.fit_transform(X)

# 5. Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=RANDOM_STATE)


In [ ]:
# 6. Обучение классификатора
classifier = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
classifier.fit(X_train, y_train)


LogisticRegression(max_iter=1000, random_state=42)

In [ ]:
# 7. Оценка модели
y_pred = classifier.predict(X_test)


In [ ]:
# Метрики оценки
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6301020408163265

Confusion Matrix:
 [[ 14   0   2   3  41]
 [  3   0   2   2  24]
 [  4   0   8  16  67]
 [  0   0   0  19 104]
 [  1   0   1  20 453]]

Classification Report:
               precision    recall  f1-score   support

           1       0.64      0.23      0.34        60
           2       0.00      0.00      0.00        31
           3       0.62      0.08      0.15        95
           4       0.32      0.15      0.21       123
           5       0.66      0.95      0.78       475

    accuracy                           0.63       784
   macro avg       0.45      0.29      0.30       784
weighted avg       0.57      0.63      0.55       784



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
test_sample = df.iloc[y_test.index].head(10)

for idx, row in test_sample.iterrows():
    print(f"Отзыв: {row['review']}")
    print(f"Рейтинг: {row['rating']}")
    print('-' * 20)

Отзыв: منيحا
Рейтинг: 4
--------------------
Отзыв: من زمان كنت عم اشتغل بقلعة تشكن الله يرحم هديك الايام
Рейтинг: 1
--------------------
Отзыв: اللقمه الشهيه الطعم الاصيل
Рейтинг: 5
--------------------
Отзыв: حلوه. الاسعار اكتر من وسط اخر شي بيبعصك vallet parking ب 5000 ليش مابعرف
Рейтинг: 3
--------------------
Отзыв: مكان جميل
Рейтинг: 5
--------------------
Отзыв: المطعم جيد بخدماته وجودة أطعمته وتنوعها
Рейтинг: 4
--------------------
Отзыв: لقد زرت هذا المكان وقد اعجبني كتير مكان راءع 
Рейтинг: 5
--------------------
Отзыв: اكل ياخذ العقل
Рейтинг: 4
--------------------
Отзыв: اكل بيت يومي
Рейтинг: 5
--------------------
Отзыв: غنية عن التعريف وعنوان لبنان
Рейтинг: 5
--------------------


In [ ]:


test_reviews = ['غنية عن التعريف وعنوان لبنان','اكل بيت يومي' ,
                'اكل ياخذ العقل',
                'لقد زرت هذا المكان وقد اعجبني كتير مكان راءع ',
'من زمان كنت عم اشتغل بقلعة تشكن الله يرحم هديك الايام',
                'لمطعم جيد بخدماته وجودة أطعمته وتنوعها',
                ' حلوه. الاسعار اكتر من وسط اخر شي بيبعصك vallet parking ب 5000 ليش مابعرف','منيحا']

In [ ]:
# Предобработка и преобразование примеров
example_clean = [preprocess_text(review) for review in test_reviews]
example_tfidf = vectorizer.transform(example_clean)

# Предсказание рейтингов
example_predictions = classifier.predict(example_tfidf)

for review, rating in zip(test_reviews, example_predictions):
    print(f'Отзыв: {review}\nПредсказанный рейтинг: {rating}\n')


Отзыв: غنية عن التعريف وعنوان لبنان
Предсказанный рейтинг: 5

Отзыв: اكل بيت يومي
Предсказанный рейтинг: 5

Отзыв: اكل ياخذ العقل
Предсказанный рейтинг: 5

Отзыв: لقد زرت هذا المكان وقد اعجبني كتير مكان راءع 
Предсказанный рейтинг: 5

Отзыв: من زمان كنت عم اشتغل بقلعة تشكن الله يرحم هديك الايام
Предсказанный рейтинг: 5

Отзыв: لمطعم جيد بخدماته وجودة أطعمته وتنوعها
Предсказанный рейтинг: 5

Отзыв:  حلوه. الاسعار اكتر من وسط اخر شي بيبعصك vallet parking ب 5000 ليش مابعرف
Предсказанный рейтинг: 5

Отзыв: منيحا
Предсказанный рейтинг: 5

